In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

# https://github.com/BNL-DAQ-LDRD/calotrack_tree/tree/main
# enum detid {
#     mvtxId = 0,
#     inttId = 1,
#     tpcId = 2,
#     tpotId = 3,
#     cemcId = 4,
#     ihcalId = 5,
#     ohcalId = 6,
#     epdId = 7,
#     mbdId = 8
# };

# input hits in calarometer is "nHits_", and in trackers "reco_cluster_"
# groundtruth hits in calarometer is "particle_" and in trackers "truth_cluster_"

In [ ]:
data_file = "./data/ppmb100evt_2.root" # sample data shared by Shuhang containing 100 events
with uproot.open(data_file) as fp:
    print(list(fp['T;1'].keys()))
data = uproot.open(data_file)["T;1"]
detid = {"mvtx": 0,"intt": 1,"tpc": 2,"tpot": 3,"cemc": 4,"ihcal": 5,"ohcal": 6,"epd": 7,"mbd": 8 };
id2det = {x:str(x)+'-'+y for y,x in detid.items()}

In [ ]:
## combine all hits and clusters per event
def get_event(event_id):
    calx = data["Hit_x"].array(library="np")[event_id]
    caly = data["Hit_y"].array(library="np")[event_id]
    calz = data["Hit_z"].array(library="np")[event_id]
    calE = data["Hit_E"].array(library="np")[event_id]
    calD = data["Hit_detid"].array(library="np")[event_id]
    trkx = data["reco_cluster_x"].array(library="np")[event_id]
    trky = data["reco_cluster_y"].array(library="np")[event_id]
    trkz = data["reco_cluster_z"].array(library="np")[event_id]
    trkE = data["reco_cluster_E"].array(library="np")[event_id]
    trkD = data["reco_cluster_detid"].array(library="np")[event_id]
    x = np.concatenate([trkx, calx])
    y = np.concatenate([trky, caly])
    z = np.concatenate([trkz, calz])
    E = np.concatenate([trkE, calE])
    D = np.concatenate([trkD, calD])
    df = pd.DataFrame({"x": x, "y": y, "z": z, "E": E, "D": D})
    df["detector type"] = df["D"].apply(lambda x: id2det[x])
    return df

In [ ]:
event_id = 9  # pick different events
## Pie Chart of number of "hits" and "clusters" from different detectors
df = get_event(event_id)
tmp = df["detector type"].to_list();
a, b= np.unique(tmp, return_counts=True)
tmp_df = pd.DataFrame({'name': a, 'count': b})
fig = px.pie(tmp_df, values='count', names='name')
fig.update_layout(legend_traceorder="normal")
fig.show()

In [ ]:
## Interactive 3D Scatter. 
## try: single click or double click the detector type in the legend.

fig = px.scatter_3d(df, x='x', y='y', z='z', color='detector type', opacity=0.3)#, symbol='D')
fig.update_traces(marker_size=3)
fig.update_layout(title=f"event {event_id}", width=800, height=800)
camera = dict(
    up=dict(x=1, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
)
fig.update_layout(scene_camera=camera, title=f"EventID={event_id}")
fig.update_layout(legend_traceorder="normal")
fig.show()